In [1]:
from kani import Kani
from kani.engines.huggingface import HuggingEngine
import glob
import os
import json
from os.path import join as pjoin
from alfworld.info import ALFWORLD_DATA
from alfworld.agents.utils.misc import add_task_to_grammar
from alfworld.agents.environment.alfred_tw_env import AlfredExpert, AlfredDemangler, AlfredExpertType
import textworld
import textworld.gym


# Alfworld

In [2]:
ALFWORLD_DATA

'/home/yl3427/.cache/alfworld'

In [5]:
### ========= Alfworld =========
# choose a problem to solve
problems = glob.glob(pjoin(ALFWORLD_DATA, "**", "initial_state.pddl"), recursive=True)
problems = [p for p in problems if "movable_recep" not in p]
len(problems)

6025

In [23]:
problems[3]

'/home/yl3427/.cache/alfworld/json_2.1.1/valid_unseen/pick_two_obj_and_place-CD-None-Safe-308/trial_T20190907_051013_060265/initial_state.pddl'

In [6]:
problem = os.path.dirname(problems[3])
print(problem)

/home/yl3427/.cache/alfworld/json_2.1.1/valid_unseen/pick_two_obj_and_place-CD-None-Safe-308/trial_T20190907_051013_060265


In [7]:
domain = pjoin(ALFWORLD_DATA, "logic", "alfred.pddl")
grammar = pjoin(ALFWORLD_DATA, "logic", "alfred.twl2")

print(domain)
print(grammar)

GAME_LOGIC = {
        "pddl_domain": open(domain).read(),
        "grammar": open(grammar).read(),
    }

# load state and trajectory files
pddl_file = os.path.join(problem, 'initial_state.pddl') # pf
json_file = os.path.join(problem, 'traj_data.json')
with open(json_file, 'r') as f:
    traj_data = json.load(f)
GAME_LOGIC['grammar'] = add_task_to_grammar(GAME_LOGIC['grammar'], traj_data)

/home/yl3427/.cache/alfworld/logic/alfred.pddl
/home/yl3427/.cache/alfworld/logic/alfred.twl2


In [8]:
gamedata = dict(**GAME_LOGIC, pddl_problem=open(pddl_file).read())
gamefile = os.path.join(os.path.dirname(pddl_file), 'game.tw-pddl')

In [32]:
gamedata

{'pddl_domain': ";; Specification in PDDL of the Alfred domain\n;; Intended to be used with Fast Downward which supports PDDL 2.2 level 1 plus the :action-costs requirement from PDDL 3.1.\n\n(define (domain alfred)\n (:requirements\n    :adl\n    :action-costs\n    :typing\n )\n (:types\n  agent\n  location\n  receptacle\n  object\n  rtype\n  otype\n  )\n\n\n (:predicates\n    (atLocation ?a - agent ?l - location)                     ; true if the agent is at the location\n    (receptacleAtLocation ?r - receptacle ?l - location)      ; true if the receptacle is at the location (constant)\n    (objectAtLocation ?o - object ?l - location)              ; true if the object is at the location\n    (openable ?r - receptacle)                                ; true if a receptacle is openable\n    (opened ?r - receptacle)                                  ; true if a receptacle is opened\n    (inReceptacle ?o - object ?r - receptacle)                ; object ?o is in receptacle ?r\n    (isRecep

In [33]:
json.dump(gamedata, open(gamefile, "w")) #textworld register

In [34]:
# expert = AlfredExpert(expert_type=AlfredExpertType.PLANNER)
expert = AlfredExpert(expert_type=AlfredExpertType.HANDCODED)

In [35]:
request_infos = textworld.EnvInfos(
    won=True,
    admissible_commands=True,
    score=True,
    max_score=True,
    intermediate_reward=True,
    extras=["expert_plan"]
)
# reset environment starts here!
env_id = textworld.gym.register_game(
    gamefile,
    request_infos,
    max_episode_steps=1000000,
    wrappers=[AlfredDemangler(), expert]
)
env = textworld.gym.make(env_id)

obs, infos = env.reset()

In [36]:
print(obs)

-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 2, a desk 1, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.

Your task is to: put two cd in safe.


In [37]:
infos

{'intermediate_reward': None,
 'won': False,
 'admissible_commands': ['go to bed 1',
  'go to desk 1',
  'go to desk 2',
  'go to drawer 1',
  'go to drawer 2',
  'go to drawer 3',
  'go to drawer 4',
  'go to drawer 5',
  'go to drawer 6',
  'go to garbagecan 1',
  'go to laundryhamper 1',
  'go to safe 1',
  'go to shelf 1',
  'go to shelf 2',
  'go to shelf 3',
  'go to shelf 4',
  'go to shelf 5',
  'go to shelf 6',
  'help',
  'inventory',
  'look'],
 'score': None,
 'facts': [Proposition('objectatlocation', (Variable('keychain 2', 'object'), Variable('loc 23', 'location'))),
  Proposition('objectatlocation', (Variable('pen 3', 'object'), Variable('loc 23', 'location'))),
  Proposition('objectatlocation', (Variable('laptop 2', 'object'), Variable('loc 13', 'location'))),
  Proposition('objectatlocation', (Variable('cd 2', 'object'), Variable('loc 23', 'location'))),
  Proposition('objectatlocation', (Variable('pen 2', 'object'), Variable('loc 4', 'location'))),
  Proposition('obje

In [53]:
init_obs = obs.split('\n')[2]
print(init_obs)
goal = obs.split('\n')[-1]
valid_actions = infos["admissible_commands"]
valid_actions.remove('look')
valid_actions.remove('inventory')
valid_actions.remove('help')

You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 2, a desk 1, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.


In [54]:
print(goal)

Your task is to: put two cd in safe.


In [55]:
obs, reward, done, infos = env.step('go to shelf 5')

In [56]:
obs

'You arrive at shelf 5. On the shelf 5, you see a keychain 1.'

In [57]:
reward

0

In [59]:
infos

{'won': False,
 'max_score': None,
 'admissible_commands': ['examine shelf 5',
  'go to bed 1',
  'go to desk 1',
  'go to desk 2',
  'go to drawer 1',
  'go to drawer 2',
  'go to drawer 3',
  'go to drawer 4',
  'go to drawer 5',
  'go to drawer 6',
  'go to garbagecan 1',
  'go to laundryhamper 1',
  'go to safe 1',
  'go to shelf 1',
  'go to shelf 2',
  'go to shelf 3',
  'go to shelf 4',
  'go to shelf 6',
  'help',
  'inventory',
  'look',
  'take keychain 1 from shelf 5'],
 'facts': [Proposition('cancontain', (Variable('drawertype ', 'rtype'), Variable('pentype ', 'otype'))),
  Proposition('cancontain', (Variable('safetype ', 'rtype'), Variable('keychaintype ', 'otype'))),
  Proposition('cancontain', (Variable('shelftype ', 'rtype'), Variable('pentype ', 'otype'))),
  Proposition('cancontain', (Variable('desktype ', 'rtype'), Variable('pentype ', 'otype'))),
  Proposition('cancontain', (Variable('drawertype ', 'rtype'), Variable('creditcardtype ', 'otype'))),
  Proposition('can

In [8]:
import requests
def run_solver(domain_file, problem_file, solver = "dual-bfws-ffparser"):

    req_body = {"domain" : domain_file, "problem" : problem_file}

    # Send job request to solve endpoint
    solve_request_url=requests.post(f"https://solver.planning.domains:5001/package/{solver}/solve", json=req_body).json()
    print(solve_request_url)
    # Query the result in the job
    celery_result=requests.post('https://solver.planning.domains:5001' + solve_request_url['result'])
    print(celery_result.json())

    while celery_result.json().get("status","")== 'PENDING':
        # Query the result every 0.5 seconds while the job is executing
        celery_result=requests.post('https://solver.planning.domains:5001' + solve_request_url['result'])

    result = celery_result.json()['result']
    return result


In [1]:
# read domain.pddl
import os


with open("domain.pddl", "r") as f:
    domain_pddl = f.read()
with open("problem.pddl", "r") as f:
    problem_pddl = f.read()

In [9]:
run_solver(domain_pddl, problem_pddl)

{'result': '/check/10c0c439-b1ed-46c6-ba06-b478ea95df51?external=True'}
{'result': {'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': ''}, 'output_type': 'generic', 'stderr': "\ndomain: syntax error in line 4, ':PRECONDITIONS':\ndomain definition expected\n", 'stdout': ''}, 'status': 'ok'}


{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan',
 'output': {'plan': ''},
 'output_type': 'generic',
 'stderr': "\ndomain: syntax error in line 4, ':PRECONDITIONS':\ndomain definition expected\n",
 'stdout': ''}

: 

# CoinCollector

In [2]:
import sys
# print(sys.path)
import time
from datetime import date
import csv
import json
import asyncio
import re

import subprocess
import requests

import os
import json
import glob
import random
import argparse
from os.path import join as pjoin

from textworld_express import TextWorldExpressEnv
import textworld
import textworld.gym

from alfworld.info import ALFWORLD_DATA
from alfworld.agents.utils.misc import add_task_to_grammar
from alfworld.agents.environment.alfred_tw_env import AlfredExpert, AlfredDemangler, AlfredExpertType

from openai import OpenAI

from pprint import PrettyPrinter
import shutil
width = shutil.get_terminal_size().columns
pp = PrettyPrinter(indent=2, width=width, compact=False, sort_dicts=False)

In [3]:
ENV_PARAMS = {"gameName": "coin", "gameParams": "numLocations=11,numDistractorItems=0,includeDoors=1,limitInventorySize=0"}
env = TextWorldExpressEnv(envStepLimit=100)
env.load(**ENV_PARAMS)

for seed in [0, 1, 2, 4, 5, 6, 7, 8, 10, 11]:
    obs, infos = env.reset(seed=seed, gameFold="train", generateGoldPath=True)
    print(f"SEED {seed}, GROUND TRUTH ACTION SEQUENCE: {list(env.getGoldActionSequence())}")

    for action in list(env.getGoldActionSequence()):
        print(f"Taking action: {action} ---->")
        obs, reward, done, infos = env.step(action)
        # print(f"Observation: {obs}") # 이거랑 Infos['observation']랑 똑같음
        print(f"Reward: {reward}") # 이것도 done이랑 중복되는 정보. 0이면 False, 1이면 True
        # print(f"Done: {done}")
        # print(f"Info: {infos}")
        # print(infos['validActions'])
        print("Info:")
        pp.pprint(infos)

SEED 0, GROUND TRUTH ACTION SEQUENCE: ['look around', 'open door to north', 'open door to east', 'move north', 'open door to west', 'move south', 'move west', 'open door to north', 'open door to south', 'open door to west', 'move east', 'move north', 'move west', 'move south', 'move south', 'open door to west', 'move west', 'open door to north', 'move north', 'move west', 'open door to west', 'move west', 'take coin']
Taking action: look around ---->
Reward: 0.0
Info:
{ 'observation': 'You are in the kitchen. In one part of the room you see a '
                 'stove. There is also an oven. You also see a fridge that is '
                 'closed. In another part of the room you see a counter, that '
                 'has nothing on it. In one part of the room you see a kitchen '
                 'cupboard that is closed. There is also a cutlery drawer that '
                 'is closed. You also see a trash can that is closed. In '
                 'another part of the room you see a

In [6]:
ENV_PARAMS = {"gameName": "coin", "gameParams": "numLocations=11,numDistractorItems=0,includeDoors=1,limitInventorySize=0"}
env = TextWorldExpressEnv(envStepLimit=100)
env.load(**ENV_PARAMS)

seed = 10
obs, infos = env.reset(seed=seed, gameFold="train", generateGoldPath=True)
pp.pprint(obs)
pp.pprint(infos['validActions'])

('You are in the kitchen. In one part of the room you see a stove. There is '
 'also an oven. You also see a fridge that is closed. In another part of the '
 'room you see a counter, that has nothing on it. In one part of the room you '
 'see a kitchen cupboard that is closed. There is also a cutlery drawer that '
 'is closed. You also see a trash can that is closed. In another part of the '
 'room you see a dishwasher that is closed. In one part of the room you see a '
 'dining chair, that has nothing on it. \n'
 'To the South you see a closed plain door. To the West you see the corridor. ')
[ 'open door to south',
  'inventory',
  'move south',
  'close door to south',
  'move west',
  'look around']


In [7]:
for action in ['move west']:
    obs, reward, done, infos = env.step(action)
    print(f"Reward: {reward}")
    pp.pprint(infos)

Reward: 0.0
{ 'observation': 'You are in the corridor. In one part of the room you see a '
                 'key holder, that has nothing on it. There is also a shoe '
                 'cabinet that is closed. You also see a umbrella stand, that '
                 'has nothing on it. In another part of the room you see a hat '
                 'rack, that has nothing on it. In one part of the room you '
                 'see a coat hanger, that has nothing on it. \n'
                 'To the North you see a closed wood door. To the South you '
                 'see a closed sliding patio door. To the East you see the '
                 'kitchen. To the West you see the living room. ',
  'look': 'You are in the corridor. In one part of the room you see a key '
          'holder, that has nothing on it. There is also a shoe cabinet that '
          'is closed. You also see a umbrella stand, that has nothing on it. '
          'In another part of the room you see a hat rack, that has noth

In [5]:
for action in ['open door to south', ]:
    obs, reward, done, infos = env.step(action)
    print(f"Reward: {reward}")
    pp.pprint(infos)

Reward: 0.0
{ 'observation': 'You open the wood door, revealing the bedroom. ',
  'look': 'You are in the living room. In one part of the room you see a '
          'wastepaper basket that is closed. There is also a book case, that '
          'has nothing on it. You also see a TV stand, that has nothing on it. '
          'In another part of the room you see a arm chair, that has nothing '
          'on it. In one part of the room you see a side table, that has '
          'nothing on it. There is also a coffee table, that has nothing on '
          'it. You also see a end table, that has nothing on it. In another '
          'part of the room you see a sofa, that has nothing on it. In one '
          'part of the room you see a coin. \n'
          'Through an open wood door, to the South you see the bedroom. '
          'Through an open wood door, to the East you see the bathroom. ',
  'inventory': 'Inventory: \n  Your inventory is currently empty.\n',
  'validActions': [ 'close door

In [22]:
env.getGoldActionSequence()

['look around',
 'open door to north',
 'open door to east',
 'move east',
 'move west',
 'move west',
 'open door to north',
 'open door to south',
 'open door to west',
 'move north',
 'open door to east',
 'move south',
 'move south',
 'open door to west',
 'move west',
 'open door to north',
 'move north',
 'move west',
 'open door to west',
 'move east',
 'move south',
 'move north',
 'move west',
 'move west',
 'take coin']

In [21]:
for action in list(env.getGoldActionSequence())[1:]:
    print(f"Taking action: {action} ---->")
    obs, reward, done, infos = env.step(action)
    print(f"Observation: {obs}")
    print(f"Reward: {reward}")
    print(f"Done: {done}")
    # print(f"Info: {infos}")
    # print(infos['validActions'])

Taking action: open door to north ---->
Observation: You open the wood door, revealing the laundry room. 
Reward: 0.0
Done: False
Taking action: open door to east ---->
Observation: You open the frosted-glass door, revealing the pantry. 
Reward: 0.0
Done: False
Taking action: move east ---->
Observation: You are in the pantry. In one part of the room you see a folding chair, that has nothing on it. There is also a shelf, that has nothing on it. 
Through an open frosted-glass door, to the West you see the kitchen. 
Reward: 0.0
Done: False
Taking action: move west ---->
Observation: You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is clo

In [ ]:
obs # 결국 문 말고는 다른걸 여는 선택지는 없네(drawer, cupboard, trashcan, fridge, ...etc.)

'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor. '

In [55]:
reward

0.0

In [45]:
done

True

In [140]:
infos

{'observation': 'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor. ',
 'look': 'You are in the bedroom. In one part of the room you see a dressing table, that has nothing on it. There is also a desk chair, that has nothing on it. You also see a desk, that has nothing on it. In another part of the room you see a chest of drawers that is closed. In one part of the room you see a wardrobe that is closed. There is also a night stand, that has nothing on it. You also see a bed, that has nothing on it. \nThrough an open wood door, to the South you see the corridor

In [68]:
env.getRunHistory()['history']

[{'observation': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room you see a dishwasher that is closed. In one part of the room you see a dining chair, that has nothing on it. There is also a coin. \nTo the South you see the corridor. To the East you see a closed plain door. ',
  'look': 'You are in the kitchen. In one part of the room you see a stove. There is also an oven. You also see a fridge that is closed. In another part of the room you see a counter, that has nothing on it. In one part of the room you see a kitchen cupboard that is closed. There is also a cutlery drawer that is closed. You also see a trash can that is closed. In another part of the room

In [ ]:
valid_actions = sorted(infos['validActions'])
valid_actions.remove('look around') # 이거 왜뺐지
valid_actions.remove('inventory') # 이거 왜뺐지

print(f"Observations: {obs} \n") 
print(f"Gold path: {env.getGoldActionSequence()} \n")
print(f"Valid Actions: {valid_actions} \n")
print(f"taskDescription: {infos['taskDescription']} \n")

# actions = ['open door to south', 'move south', 'open door to west', 'move west', 'move east', 'move north', 'open door to west', 'move west', 'move east', 'move south', 'move south', 'move north', 'move east', 'open door to north', 'move north', 'take coin']
actions = ['open door to south']

for action in actions:
    print('>', action)
    obs, reward, done, infos = env.step(action)
    print(obs)

print(infos['done'])

In [4]:
domain_file = open(f'/local-ssd/yl3427/hazlab-starter-code/data/test/domain.pddl').read()
problem_file = open(f'/local-ssd/yl3427/hazlab-starter-code/data/test/problem.pddl').read()
domain_file

' (define (domain explore-env)\n  (:requirements :strips :typing)\n  (:types location direction)\n  (:predicates \n    (at ?loc - location)\n    (door-open ?from - location ?to - location ?dir - direction)\n    (connected ?from - location ?to - location ?dir - direction)\n  )\n\n  (:action open-door\n    :parameters (?loc1 - location ?loc2 - location ?dir - direction)\n    :precondition (and (connected ?loc1 ?loc2 ?dir))\n    :effect (door-open ?loc1 ?loc2 ?dir)\n  )\n\n  (:action move\n    :parameters (?from - location ?to - location ?dir - direction)\n    :precondition (and (at ?from) (connected ?from ?to ?dir) (door-open ?from ?to ?dir))\n    :effect (and (not (at ?from)) (at ?to))\n  )\n) '

In [5]:
max_retries=3

req_body = {"domain" : domain_file, "problem" : problem_file}

retries = 0

while retries < max_retries:
    try:
        # Send job request to solve endpoint
        solve_request_url = requests.post(
            f"https://solver.planning.domains:5001/package/dual-bfws-ffparser/solve", 
            json=req_body
        ).json()

        print(solve_request_url)

        # Query the result in the job
        celery_result = requests.post(
            'https://solver.planning.domains:5001' + solve_request_url['result']
        )
        print(celery_result.json().get("status", "?"))
        while celery_result.json().get("status", "") == 'PENDING':
            time.sleep(0.5)
            celery_result = requests.post(
                'https://solver.planning.domains:5001' + solve_request_url['result']
            )
        
        print(celery_result.json()['result'])
        break

    except Exception as e:
        print(f"Error encountered: {e}. Retrying in 5 seconds... (Attempt {retries+1}/{max_retries})")
        retries += 1
        time.sleep(5)

{'result': '/check/9553993b-3e09-4104-acdc-22a818cf7478?external=True'}
ok
{'call': 'timeout 30 planutils run dual-bfws-ffparser -- domain problem plan', 'output': {'plan': '(OPEN-DOOR STREET UNKNOWN-NORTH NORTH)\n(MOVE STREET UNKNOWN-NORTH NORTH)\n'}, 'output_type': 'generic', 'stderr': '', 'stdout': ' --- OK.\n Match tree built with 18 nodes.\n\nPDDL problem description loaded: \n\tDomain: EXPLORE-ENV\n\tProblem: EXPLORE-ENV-PROBLEM\n\t#Actions: 18\n\t#Fluents: 12\nGoals found: 1\nGoals_Edges found: 1\nStarting search with 1-BFWS...\n--[1 / 0]--\n--[1 / 1]--\n--[0 / 0]--\n--[0 / 2]--\nTotal time: 0.000242999\nNodes generated during search: 8\nNodes expanded during search: 4\nPlan found with cost: 2\nFast-BFS search completed in 0.000242999 secs\n'}


In [6]:
celery_result.json()['result']['output']

{'plan': '(OPEN-DOOR STREET UNKNOWN-NORTH NORTH)\n(MOVE STREET UNKNOWN-NORTH NORTH)\n'}

In [12]:
def map_actions(action):
    actions = action.lower().replace("(", "").replace(")", "").split('\n')
    action_lst = []
    for act in actions:
        if "open" in act and "door" in act:
            direction = act.split(' ')[-1]
            action_lst.append(f'open door to {direction}')
        elif "move" in act:
            action_lst.append(f"move {act.split(' ')[-1]}")
    if len(action_lst) == 0:
        return None
    return action_lst

In [13]:
map_actions(celery_result.json()['result']['output']['plan'])

['open door to west', 'move west']

In [15]:
"'hello".startswith("'")

True